In [21]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split


# Đọc dữ liệu từ tệp JSON
def load_json_data(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    return data


# Chuẩn bị dữ liệu
def prepare_data(data):
    t = np.array(data["t"])
    o = np.array(data["o"])
    h = np.array(data["h"])
    l = np.array(data["l"])
    c = np.array(data["c"])
    v = np.array(data["v"])

    features = np.column_stack((o, h, l, c, v))
    labels = features.copy()  # Đầu ra là các giá trị o, h, l, c, v
    return features, labels


# Xây dựng mô hình LSTM
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=5, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=5))
    model.add(
        Dense(units=5)
    )  # Số lượng units là 5 vì chúng ta có 5 đầu ra: o, h, l, c, v
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


# Đào tạo mô hình
def train_model(model, X_train, y_train, epochs=1, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


# Dự đoán với mô hình
def predict(model, X_test):
    predictions = model.predict(X_test)
    return predictions

In [22]:
# Đọc dữ liệu từ tệp JSON và chuẩn bị dữ liệu
data = load_json_data('data.json')
X, y = prepare_data(data)
n_features = X.shape[1]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Xây dựng và đào tạo mô hình
model = build_lstm_model((1, n_features))
train_model(model, X_train, y_train)

# Lấy dữ liệu cuối cùng từ data.json
last_data = X[-1].reshape((1, 1, n_features))

# Dự đoán ngày mai
tomorrow_prediction = predict(model, last_data)

# In dự đoán cho ngày mai
print("Predicted values for tomorrow:")
print("Open:", tomorrow_prediction[0][0])
print("High:", tomorrow_prediction[0][1])
print("Low:", tomorrow_prediction[0][2])
print("Close:", tomorrow_prediction[0][3])
print("Volume:", tomorrow_prediction[0][4])


1/1 [==============================] - 2s 2s/step
Predicted values for tomorrow:
Open: 0.02111103


IndexError: index 1 is out of bounds for axis 0 with size 1